In [28]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
import csv

In [29]:
path_drep = "../drep_genomes/OUTPUT/rep_genomes/"
drep_samples = {}
for directory in os.listdir(path_drep):
    path_2 = path_drep + directory
    if os.path.isdir(path_2):
        for directory_2 in os.listdir(path_2):
            path_3 = path_2 + "/" + directory_2
            if os.path.isdir(path_3):
                for directory_3 in os.listdir(path_3):
                    path_4 = path_3 + "/" + directory_3 + "/"
                    if os.path.isdir(path_4):
                        for directory_samp in os.listdir(path_4):
                            samp_dir = path_4 + directory_samp
                            if os.path.isdir(samp_dir):
                                drep_samples[directory_samp] = path_4

In [30]:
def get_mmseqs_input(file_paths_list, outfile_path):
    num_samples = len(file_paths_list)
    final_cycle_ind = num_samples //1000
    cat_cmd_1 = "cat "
    for i in range(1000):
        cat_cmd_1 += file_paths_list[i] + " "
    cat_cmd_1 += " > " + outfile_path
    os.system(cat_cmd_1)

    for i in range(1, final_cycle_ind):
        cat_cmd = "cat "
        for j in range(i * 1000, (i + 1) * 1000):

            cat_cmd += file_paths_list[j] + " "
        cat_cmd += " >> " + outfile_path
        os.system(cat_cmd)

    cat_cmd = "cat "
    for i in range(final_cycle_ind * 1000, num_samples):
        cat_cmd += file_paths_list[i] + " "
    cat_cmd += " >> " + outfile_path
    os.system(cat_cmd)

### make multi prodigal .faa inputs

In [5]:
prodigal_paths = []
for sample in drep_samples.keys():
    sample_path = drep_samples[sample] + sample
    prodigal_path = sample_path + "/" + sample + ".prodigal.faa.gz"
    prodigal_paths.append(prodigal_path)
    
outfile_path = "../clusters/INPUT/mmseqs2_testdb_input.faa.gz"

#get_mmseqs_input(prodigal_paths, outfile_path)

In [6]:
backup_cmd = "gsutil cp " + outfile_path + " gs://jluo_bucket/ggdb/clusters"
#os.system(backup_cmd)

### make multi .gff inputs

In [7]:
print("making multi .gff inputs")
gff_paths = []
for sample in drep_samples.keys():
    sample_path = drep_samples[sample] + sample
    prodigal_path = sample_path + "/" + sample + ".prodigal.gff.gz"
    prodigal_paths.append(prodigal_path)
    
outfile_path_gff = "../clusters/mmseqs2_testdb_input.gff.gz"

#get_mmseqs_input(prodigal_paths, outfile_path_gff)

making multi .gff inputs


In [ ]:
backup_cmd_2 = "gsutil cp " + outfile_path_gff + " gs://jluo_bucket/ggdb/clusters"
#os.system(backup_cmd_2)

### data exploration of failed samples

In [35]:
path_temp = "ggdb_multisql_errorlog.csv"

In [47]:
in_df = pd.read_csv(path_temp, header=None)
in_df = in_df.iloc[3928:].set_index(0)
#in_df.to_csv(path_temp, header = False)

## choose representatives for stringent clusters

In [31]:
# iterate through unique pid (cluster representatives) in tsv
# explore data (distribution of cluster sizes, do any representatives have >1 *)
# search for protein in fasta file (load in memory for now)
# length + # of *'s
# priority is (1) no * (2) length 

# 1. make protein_stats csv file that summarizes each protein's viability as a representataive of a cluster

path_coords = "../clusters/proteines_testdb_10k.faa"

outfile_stats = open("../clusters/proteins_stats_perm_clustering.csv", "w")
print("pid,isgapfree,length,iscomplete,sequence", file=outfile_stats)
               
fasta_sequences = SeqIO.parse(open(path_coords),'fasta')
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    p_gapfree = sequence[:-1].count("*") == 0
    p_len = len(sequence)
    p_complete = (sequence[0] == "M")&(sequence[-1] == "*")
    print(name + "," + str(p_gapfree) + "," + str(p_len) + "," + str(p_complete) + "," + sequence, file=outfile_stats)

#gff_sequences = SeqIO.parse(open()    
    
# 2. make pid to fragment dictionary
#pid_to_frag    

In [32]:
outfile_stats_df = pd.read_csv("../clusters/proteins_stats_perm_clustering.csv")

In [45]:
#outfile_stats_df[outfile_stats_df["length"] < 100]

In [34]:
path_tsv = "../clusters/OUTPUT/stringent/tmp/clu_cluster_10k.tsv"
with open(path_tsv, "r") as in_file:
    for line in csv.reader(in_file, delimiter='\t'):
        # search for protein
        # get stats
        # write to new file
        unique_pid = line[1]
    

In [46]:
pd.read_csv(path_tsv, sep = "\t", header = None)

,0,1
0,8d08b269589e6b17da1953549c42646eb974e87a2acdea...,0bb0f09df534faa350cde83aeee781cf36778f82b310a9...
1,8d08b269589e6b17da1953549c42646eb974e87a2acdea...,0bb0f09df534faa350cde83aeee781cf36778f82b310a9...
2,8d08b269589e6b17da1953549c42646eb974e87a2acdea...,e0b63cf53261c8a6484fb78e7fb45a53775928c4f3411d...
3,8d08b269589e6b17da1953549c42646eb974e87a2acdea...,8e8213f5a2e431aa9999d96d1e4c707b5c83eeb2527c56...
4,8d08b269589e6b17da1953549c42646eb974e87a2acdea...,b908dfca9078e34c86a9d8f576c6d9a69e9f1cf98e3575...
...,...,...
9896,8bc4bbe4b8150dc6efdbb0a5eb5e61460c23a369eeba74...,8bc4bbe4b8150dc6efdbb0a5eb5e61460c23a369eeba74...
9897,4f74bedafe2406184579f4a3c3ecaef508c9a05958c63a...,4f74bedafe2406184579f4a3c3ecaef508c9a05958c63a...
9898,db4c8682a3627478f39c7a9f4d07ba9c61e76d77227ddb...,db4c8682a3627478f39c7a9f4d07ba9c61e76d77227ddb...
9899,83609878f820f061c41523cc3c82add7d7e3ab3934fd7f...,83609878f820f061c41523cc3c82add7d7e3ab3934fd7f...


In [49]:
unique_pid = pd.read_csv(path_tsv, sep = "\t", header = None)[1].unique()

In [50]:
len(unique_pid)

9208

## Cluster_df data ex to get cluster summary stats

In [22]:
cluster_df = pd.read_csv(path_tsv, sep = '\t', header = None)
rep_counts = cluster_df.groupby(0).count()[1]

In [31]:
all_pids_s = cluster_df[1]